In [20]:
import csv
import sys
import re
import matplotlib.pyplot as plt
import numpy as np

# Cutoff for rare variants 

In [21]:
cutoff  = 0.0000001

In [22]:
def writelist (mylist, myfile): 
    fp =  open(myfile,'a')
    for idx,item in enumerate(mylist):
            fp.write(item)
            fp.write('\t')
    fp.write('\n')
    fp.close()

In [23]:
def intersections(list1, list2):
    intersection_list = []
    for i in list1: 
        if i in list2: 
                intersection_list.append(i)
    return intersection_list

In [24]:
def difference(list1, list2):
    unique_list = []
    for i in list1: 
        if i not in list2: 
                unique_list.append(i)
    return unique_list

In [25]:
def exist (list1, list2): 
    found_list = []
    for i in list1: 
            if i not in found_list: 
                    found_list.append(i)
    for i in list2: 
            if i not in found_list: 
                    found_list.append(i)
    return found_list 

In [26]:
def common_variants(variants1, variants2, outfile): 
        fp = open(outfile, 'w')
        for i in variants1: 
               for j in variants2: 
                     count = 0 
                     for a in range(0,4): 
                            if i[a] == j[a]:
                                count +=1
                     if count == 4: 
                          for item in (i):
                                fp.write(item)
                                fp.write('\t')
                          fp.write('\n')

# Read annovar annotation file

In [27]:
 def load_annovar(filename):
    rows = []
    exacall = []
    gnomall = []
    rs = []
    gene = []
    pathogenics = []
    functions = []
    exonicfunctions =[]
    fp = open(filename, 'r')
    r = csv.reader(fp, delimiter='\t')
    for n, row in enumerate(r):
        if n == 0: 
                #for i in range(0,len(row) ): 
                #     print (i,row[i])
                print(row[7])
                continue
        if n == 100:
               #for i in range(0,len(row) ): 
                      #print (i,row[i])
                print(row[7])
                continue
        exonicfunctions.append(row[5])
        exacall.append(row[16]) #old is 11
        gnomall.append(row[124]) #old is 119
        rs.append(row[24]) #old was 19
        rows.append(row)
        gene.append(row[6]) 
        functions.append(row[5]) #old was 8
        pathogenics.append(row[60]) #old was 54
    return rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics

# Low MAF in gnomAD or ExAC

In [28]:
rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics= load_annovar('74anno.hg19_multianno.txt')
count = 0
print ('Starting ....')
outfile = 'f74.raremaf.filtered.txt'
f74m1_raremaf_genes = []
f74m1_raremaf_variants = [] 
print (cutoff)
for i in range(0,len(exacall) ): 
    #print(gnomall[i], exacall[i],functions[i])
    if ( (gnomall[i] != '.') and   (exacall[i] !='.' ) ) :
        if ( (   (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff)   )   and (functions[i] =='exonic')  ): 
            count += 1 
            writelist(rows[i], outfile)
            print(rs[i], gene[i], exacall[i],gnomall[i])
            if gene[i] not in f74m1_raremaf_genes: 
                    f74m1_raremaf_genes.append(gene[i])
            f74m1_raremaf_variants.append(rows[i])
            #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]        
            #if var not in f74m1_raremaf_variants: 
                #f74m1_raremaf_variants.append(var)
    elif ( (gnomall[i]=='.') and (exacall[i].isdigit())):
           if ((float(exacall[i]) < cutoff)and (functions[i] =='exonic')  ):
                count += 1 
                writelist(rows[i], outfile)
                print(rs[i], gene[i], exacall[i],gnomall[i])
                if gene[i] not in f74m1_raremaf_genes: 
                        f74m1_raremaf_genes.append(gene[i])
                f74m1_raremaf_variants.append(rows[i])
                #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]
                #if var not in f74m1_raremaf_variants: 
                    #f74m1_raremaf_variants.append(var)
           
    elif (( exacall[i]=='.') and   (gnomall[i].isdigit())): 
            if( (float (gnomall[i]) < cutoff)  and (functions[i] =='exonic')  ):
                count += 1 
                writelist(rows[i], outfile)
                print(rs[i], gene[i], exacall[i],gnomall[i])
                if gene[i] not in  f74m1_raremaf_genes: 
                         f74m1_raremaf_genes.append(gene[i])
                f74m1_raremaf_variants.append(rows[i])       
                #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]
                #if var not in f74m1_raremaf_variants: 
                    #f74m1_raremaf_variants.append(var)
print ('We reached printing stage')
print ('Number of variants with gnomall and/or exac <', cutoff, 'is:',count)
print('========================================================')
print(f74m1_raremaf_genes)
print('========================================================')
print (f74m1_raremaf_variants)
print('Done')

GeneDetail.refGene
.
Starting ....
1e-07
('.', 'PLCH2', '.', '0')
('.', 'ATPAF1', '.', '0')
('rs79475403', 'IGSF3', '.', '0')
('rs781796911', 'NBPF10;NBPF20', '0', '.')
('.', 'CGN', '.', '0')
('.', 'NPR1', '.', '0')
('.', 'KIF26B', '.', '0')
('.', 'TRIM58', '.', '0')
('.', 'ANKRD36B', '.', '0')
('.', 'ANKRD36B', '.', '0')
('.', 'ANKRD36B', '.', '0')
('.', 'ZNF142', '.', '0')
('.', 'SPEG', '.', '0')
('.', 'GALNT15', '.', '0')
('rs116300435', 'FRG2C', '0', '.')
('rs115516351', 'FRG2C', '0', '.')
('rs62247159', 'FRG2C', '0', '.')
('rs62247960', 'FRG2C', '0', '.')
('rs62247961', 'FRG2C', '0', '.')
('rs2118760', 'FRG2C', '0', '.')
('rs2118754', 'ZNF717', '.', '0')
('rs148249548', 'ZNF717', '.', '0')
('rs76921234', 'ZNF717', '0', '.')
('rs138751618', 'ZNF717', '.', '0')
('rs2918520', 'ZNF717', '.', '0')
('rs749178396', 'WWTR1', '.', '0')
('.', 'PSAPL1', '.', '0')
('.', 'CMYA5', '.', '0')
('rs12721829', 'HLA-B', '0', '.')
('rs1131233', 'HLA-B', '0', '.')
('.', 'SOBP', '.', '0')
('rs769279477'

# Filter for rare MAF

In [29]:
def maf_filter(mfile): 
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    outfile = mfile.split('.')[0] + '_raremaf.txt'
    raremaf_genes = []
    variants_list = [] 
    for i in range(0,len(exacall) ): 
        if exfunc[i] =='exonic':
                    #========================================
                    #If both gnomAD and ExAC is '.'
                    #========================================
                    if (gnomall[i] != '.') and (exacall[i] !='.'):
                        if   (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff) : 
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both gnomAD is '.' and ExAC is not '.'
                    #========================================               
                    elif (gnomall[i] == '.') and (exacall[i] != '.') and (float(exacall[i]) < cutoff) :
                            count += 1 
                            writelist(rows[i], outfile)                    
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both ExAC is '.' and gnomAD is not '.'
                    #======================================== 
                    elif (exacall[i] =='.') and (gnomall[i] != '.') and (float (gnomall[i]) < cutoff)  :
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in  raremaf_genes: 
                                     raremaf_genes.append(gene[i])
    print ('Number of variants that are rare  cutoff <', cutoff, 'is:',count)
    return(raremaf_genes,  variants_list)

# Filter for pathogenic variants

In [30]:
def pathogenic_filter(mfile):
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    pathogenic_genes = []    
    variants_list = [] 
    outfile = mfile.split('.')[0]+'pathogenic.txt'
    for i in range(0,len(exacall) ): 
        #if 'athogenic' in pathogenics[i]: 
        if 'athogenic' in rows[i]: 
                        print (rows[i])
            #if (exfunc[i] =='exonic'): 
                #if ((exacall[i] == '.') or( float(exacall[i]) < cutoff)) and  ((gnomall[i] =='.') or (float(gnomall[i]) < cutoff)):
                        count += 1 
                        writelist(rows[i], outfile)
                        variants_list.append(rows[i])
                        print(rs[i], gene[i], exfunc[i],exacall[i],gnomall[i], pathogenics[i])
                        if gene[i] not in pathogenic_genes:
                                pathogenic_genes.append(gene[i])
    print ('Number of variants that are rare and pathogenic cutoff <', cutoff, 'is:',count)
    return pathogenic_genes, variants_list 

# Filter for no MAF variants

In [31]:
def nomaf_filter(mfile):    
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    nomafs_genes = []
    nomafvariants = [] 
    outfile = mfile.split('.')[0]+'nomaf.txt'
    
    for i in range(0,len(exacall) ): 
        if (exacall[i] == '.') and (gnomall[i] =='.') and exfunc[i] =='exonic':
                count += 1 
                writelist(rows[i], outfile)
                if gene[i] not in nomafs_genes:
                        nomafs_genes.append(gene[i])
                var = rs[i]+'.'+exfunc[i]+'.'+gene[i]
                if var not in  nomafvariants: 
                         nomafvariants.append(var)
    print ('Number of variants with no MAf in both exacall and gnomAD <', cutoff, 'is:',count)
    return nomafs_genes,  nomafvariants 

# F74 Analysis

In [32]:
f74raremaf, f74_variants  = maf_filter('74anno.hg19_multianno.txt')
print('Done')


GeneDetail.refGene
.
('.', 'exonic', 'PLCH2', '.', '0')
('.', 'exonic', 'ATPAF1', '.', '0')
('rs79475403', 'exonic', 'IGSF3', '.', '0')
('rs781796911', 'exonic', 'NBPF10;NBPF20', '0', '.')
('.', 'exonic', 'CGN', '.', '0')
('.', 'exonic', 'NPR1', '.', '0')
('.', 'exonic', 'KIF26B', '.', '0')
('.', 'exonic', 'TRIM58', '.', '0')
('.', 'exonic', 'ANKRD36B', '.', '0')
('.', 'exonic', 'ANKRD36B', '.', '0')
('.', 'exonic', 'ANKRD36B', '.', '0')
('.', 'exonic', 'ZNF142', '.', '0')
('.', 'exonic', 'SPEG', '.', '0')
('.', 'exonic', 'GALNT15', '.', '0')
('rs116300435', 'exonic', 'FRG2C', '0', '.')
('rs115516351', 'exonic', 'FRG2C', '0', '.')
('rs62247159', 'exonic', 'FRG2C', '0', '.')
('rs62247960', 'exonic', 'FRG2C', '0', '.')
('rs62247961', 'exonic', 'FRG2C', '0', '.')
('rs2118760', 'exonic', 'FRG2C', '0', '.')
('rs2118754', 'exonic', 'ZNF717', '.', '0')
('rs148249548', 'exonic', 'ZNF717', '.', '0')
('rs76921234', 'exonic', 'ZNF717', '0', '.')
('rs138751618', 'exonic', 'ZNF717', '.', '0')
('rs2

In [33]:
f74_pathogenic,f74_pathogenic_variants   = pathogenic_filter('74anno.hg19_multianno.txt')

GeneDetail.refGene
.
('Number of variants that are rare and pathogenic cutoff <', 1e-07, 'is:', 0)


In [17]:
f74_nomaf = nomaf_filter('74anno.hg19_multianno.txt')

GeneDetail.refGene
.
('Number of variants with no MAf in both exacall and gnomAD <', 1e-07, 'is:', 2988)


# Find if REST exist in Patient 74

In [18]:
rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics= load_annovar('74anno.hg19_multianno.txt')
count = 0
outfile = 'f74.raremaf.filtered.txt'
f74m1_raremaf_genes = []
f74m1_raremaf_variants = [] 
for i in range(0,len(gene) ): 
      if gene[i] == 'REST': 
                      print gene[i],functions[i], gnomall[i], exacall[i]
print('Done')

GeneDetail.refGene
.
REST UTR5 . .
REST intronic . .
REST intronic . .
REST exonic 0.0112 0.0111
REST UTR3 . .
Done


In [36]:
rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics= load_annovar('74anno.hg19_multianno.txt')
for i in range(0,len(gene) ): 
      if gene[i] == 'GOLGA6L2': 
                      print gene[i],functions[i], gnomall[i], exacall[i]
                      print (rows[i])
print('Done')

GeneDetail.refGene
.
GOLGA6L2 exonic . .
['chr15', '23685008', '23685008', '-', 'ATC', 'exonic', 'GOLGA6L2', '.', 'nonframeshift insertion', 'GOLGA6L2:NM_001304388:exon8:c.2613_2614insGAT:p.D872delinsDD', 'exonic', 'ENSG00000174450', '.', 'nonframeshift insertion', 'ENSG00000174450:ENST00000345070:exon8:c.828_829insGAT:p.D277delinsDD,ENSG00000174450:ENST00000567107:exon8:c.2613_2614insGAT:p.D872delinsDD,ENSG00000174450:ENST00000312015:exon9:c.1634_1635insGAT:p.E545delinsEM', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 